In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from requests import get
import tensorflow as tf
import os

In [2]:
%load_ext tensorboard

In [3]:
tf.__version__

'2.12.0'

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [5]:
pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
pets = ['PET_AMMONITE', 'PET_ARMADILLO', 'PET_BABY_YETI', 'PET_BAL','PET_BAT','PET_BEE','PET_BLACK_CAT','PET_BLAZE','PET_BLUE_WHALE','PET_CHICKEN','PET_DOLPHIN','PET_ELEPHANT','PET_ENDER_DRAGON','PET_ENDERMAN','PET_ENDERMITE','PET_FLYING_FISH','PET_GHOUL','PET_GIRAFFE','PET_GOLDEN_DRAGON','PET_GOLEM','PET_GRIFFIN','PET_GUARDIAN','PET_HORSE','PET_HOUND','PET_JELLYFISH','PET_JERRY','PET_LION','PET_MAGMA_CUBE','PET_MEGALODON','PET_MITHRIL_GOLEM','PET_MONKEY','PET_MOOSHROOM_COW','PET_OCELOT','PET_PARROT','PET_PHOENIX','PET_PIG','PET_PIGMAN','PET_RABBIT','PET_RAT','PET_REINDEER','PET_ROCK','PET_SCATHA','PET_SHEEP','PET_SILVERFISH','PET_SKELETON','PET_SKELETON_HORSE','PET_SNAIL','PET_SNOWMAN','PET_SPIDER','PET_SPIRIT','PET_SQUID','PET_TARANTULA','PET_TIGER','PET_TURTLE','PET_FROST_WISP','PET_WITHER_SKELETON','PET_WOLF','PET_ZOMBIE']

In [7]:
petsData = []

for i in pets:
  for j in range(1):
    res = get(f'https://sky.coflnet.com/api/auctions/tag/{i}/sold?page={j}&pageSize=1000&token=64wcAGNBYE8tMW7w9W2vWq')
    petsData.extend(res.json())

In [8]:
petsData = list(filter(lambda elm: not isinstance(elm, str), petsData))

In [9]:
print(len(petsData))

56766


In [10]:
raw_df = pd.DataFrame(petsData)
raw_df.columns

Index(['uuid', 'count', 'startingBid', 'tag', 'itemName', 'start', 'end',
       'auctioneerId', 'profileId', 'coop', 'coopMembers', 'highestBidAmount',
       'bids', 'anvilUses', 'enchantments', 'nbtData', 'itemCreatedAt',
       'reforge', 'category', 'tier', 'bin', 'flatNbt'],
      dtype='object')

In [11]:
nbtData = pd.json_normalize(raw_df.flatNbt)

In [12]:
nbtData = nbtData[['exp', 'tier', 'heldItem', 'skin']]

In [13]:
nbtData.fillna('<null>', inplace=True)

In [14]:
nbtData['exp'] = nbtData['exp'].apply(float)

In [15]:
vocabulary = list(set(nbtData.tier.unique().tolist() + pets + nbtData.heldItem.unique().tolist() + nbtData.skin.unique().tolist()))

In [16]:
cat_to_id = tf.keras.layers.StringLookup(vocabulary=vocabulary, mask_token=None)

In [17]:
nbtData['tier'] = cat_to_id(nbtData['tier'])
nbtData['heldItem'] = cat_to_id(nbtData['heldItem'])
nbtData['skin'] = cat_to_id(nbtData['skin'])

In [18]:
nbtData

,exp,tier,heldItem,skin
0,1.824320e+06,92,67,27
1,0.000000e+00,92,27,27
2,0.000000e+00,92,27,27
3,6.398701e+05,92,67,27
4,2.543691e+07,92,172,27
...,...,...,...,...
56761,0.000000e+00,42,27,27
56762,2.535436e+07,92,64,27
56763,1.948773e+04,92,27,27
56764,0.000000e+00,92,27,27


In [19]:
auction_data = raw_df[['highestBidAmount', 'end', 'tag']]

In [20]:
auction_data['end'] = pd.to_datetime(auction_data['end']).astype('int64')/ 10**9

In [21]:
auction_data['tag'] = cat_to_id(auction_data['tag'])

In [22]:
combined_data = pd.concat([nbtData, auction_data], axis=1)

In [23]:
train_data = combined_data.sample(frac=.8)
train_numeric_data = train_data[['exp', 'end']]
train_categorical_data = train_data[['tier','heldItem', 'skin', 'tag']]
train_label = train_data.pop('highestBidAmount')

In [24]:
test_data = combined_data.drop(train_data.index)
test_numeric_data = test_data[['exp', 'end']]
test_categorical_data = test_data[['tier','heldItem', 'skin', 'tag']]
test_label = test_data.pop('highestBidAmount')

In [25]:
input_normalizer = tf.keras.layers.Normalization()
input_normalizer.adapt(train_numeric_data)

In [26]:
input_normalizer.mean.numpy()

array([[9.2724470e+06, 1.6893435e+09]], dtype=float32)

In [51]:
class Predictor(tf.keras.Model):
    def __init__(self):
        super().__init__(self)
        self.num_dense = tf.keras.layers.Dense(32, activation='relu')

        self.embeddings = tf.keras.layers.Embedding(len(cat_to_id.get_vocabulary()), 2)
        self.cat_dense = tf.keras.layers.Dense(32, activation='relu')
        self.flatten_layer = tf.keras.layers.Flatten()

        self.combine_layer = tf.keras.layers.Concatenate(axis=1)
        self.combined_dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.combined_dense3 = tf.keras.layers.Dense(8, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None, mask=None):
        cat_data = inputs[1]
        num_data = inputs[0]

        # cat_data = cat_to_id(cat_data)
        cat_data = self.embeddings(cat_data)
        cat_data = self.flatten_layer(cat_data)
        cat_data = self.cat_dense(cat_data)

        num_data = input_normalizer(num_data)
        num_data = self.num_dense(num_data)

        concatenated = self.combine_layer([num_data, cat_data])
        concatenated = self.combined_dense2(concatenated)
        concatenated = self.combined_dense3(concatenated)

        y = self.output_layer(concatenated)
        return y

In [60]:
with tf.device('/GPU:0'):
    model = Predictor()

In [64]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.5), loss='log_cosh', metrics=['mae', 'mape'])

In [65]:
tb_callback = tf.keras.callbacks.TensorBoard('./logs', update_freq=1)

In [ ]:
history = model.fit([train_numeric_data, train_categorical_data], train_label, epochs=20, batch_size=32, callbacks=[tb_callback])

Epoch 1/20
1420/1420 [==============================] - 6s 3ms/step - loss: 11103151.0000 - mae: 11103151.0000 - mape: 829.3501
Epoch 2/20
1420/1420 [==============================] - 5s 3ms/step - loss: 6532708.0000 - mae: 6532709.0000 - mape: 637.5681
Epoch 3/20
1420/1420 [==============================] - 5s 3ms/step - loss: 5891048.5000 - mae: 5891049.0000 - mape: 620.9203
Epoch 4/20
1420/1420 [==============================] - 5s 3ms/step - loss: 5221553.5000 - mae: 5221553.5000 - mape: 605.2407
Epoch 5/20
1420/1420 [==============================] - 5s 3ms/step - loss: 5320625.5000 - mae: 5320626.0000 - mape: 569.8608
Epoch 6/20
1098/1420 [======================>.......] - ETA: 1s - loss: 4952883.0000 - mae: 4952884.0000 - mape: 460.5309

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
plt.scatter(test_label, model.predict([test_numeric_data, test_categorical_data]))